# Imbalance Problem - Iris Dataset

`pip install imblearn`


In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

# Iris data
dataObj = load_iris()
df = pd.DataFrame(data=dataObj.data, columns=dataObj.feature_names)
df["class"] = dataObj.target
df["class"].value_counts().sort_index(ascending=True).plot(kind="bar")

Let's make one class very small in number.


In [ ]:
df0 = df[df['class'] == 2].sample(frac=0.2, random_state=1)
dfR = df[df['class'] != 2]
dfSampled = pd.concat([df0, dfR], axis=0, ignore_index=True)
dfSampled['class'].value_counts().sort_index(ascending=True).plot(kind='bar')

In [ ]:
colsX = [i for i in dfSampled.columns if i != 'class']
X = dfSampled[colsX].values
y = dfSampled['class'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
display( pd.Series(data=y_train).value_counts())
display( pd.Series(data=y_test).value_counts())

# Standardize
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# Constructing classifier
lr = LogisticRegression(random_state=0, C=1)

# Train and predict
lr.fit(X_train_std, y_train)
y_pred = lr.predict(X_test_std)

In [ ]:
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    estimator=lr, X=X_test_std, y=y_test, display_labels=dataObj.target_names
)
plt.show()

## Use the `class weight` setting


In [ ]:
# Constructing a pipeline object
lr_balanced = LogisticRegression(random_state=0, C=1, class_weight='balanced')

lr_balanced.fit(X_train_std, y_train)

y_pred = lr_balanced.predict(X_test_std)

In [ ]:
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(
    estimator=lr_balanced, X=X_test_std, y=y_test, display_labels=dataObj.target_names
)
plt.show()

## Use SMOTE


In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train_os, y_train_os = oversample.fit_resample(X_train_std, y_train)
pd.Series(y_train_os).value_counts()

In [ ]:
lr.fit(X_train_os, y_train_os)
y_pred = lr.predict(X_test_std)

In [ ]:
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(
    estimator=lr, X=X_test_std, y=y_test, display_labels=dataObj.target_names
)
plt.show()